<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Лемматизация-и-удаление-стоп-слов" data-toc-modified-id="Лемматизация-и-удаление-стоп-слов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Лемматизация и удаление стоп-слов</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#Word2vec" data-toc-modified-id="Word2vec-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Word2vec</a></span></li><li><span><a href="#BERT.-Создание-эмбеддингов" data-toc-modified-id="BERT.-Создание-эмбеддингов-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>BERT. Создание эмбеддингов</a></span></li><li><span><a href="#Подбор-гиперпараметров" data-toc-modified-id="Подбор-гиперпараметров-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Подбор гиперпараметров</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#TFIDF" data-toc-modified-id="TFIDF-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>TFIDF</a></span></li><li><span><a href="#Word2vec" data-toc-modified-id="Word2vec-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Word2vec</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Проверка-на-тесте-лучшей-модели-и-подхода" data-toc-modified-id="Проверка-на-тесте-лучшей-модели-и-подхода-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Проверка на тесте лучшей модели и подхода</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификатор токсичности комментариев

## Подготовка

In [1]:
import sys

!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
!pip install optuna

You should consider upgrading via the '/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/bin/python3.9 -m pip install --upgrade pip' command.
/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/lib/python3.9/site-packages/torch/cuda/__init__.py:529: UserWarning: HIP initialization: Unexpected error from hipGetDeviceCount(). Did you run some cuda functions before calling NumHipDevices() that might have already set an error? Error 101: hipErrorInvalidDevice (Triggered internally at ../c10/hip/HIPFunctions.cpp:110.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 12.8 MB 1.2 MB/s eta 0:00:01
You should consider upgrading via th

In [2]:
# импорт библиотек
import logging
import random
import re

import gensim
import nltk
import numpy as np
import optuna
import pandas as pd
import spacy
import torch
import transformers
import torch.nn as nn

from catboost import CatBoostClassifier
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from tqdm import notebook

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
stopwords = nltk_stopwords.words('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/troflianina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/troflianina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/troflianina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/troflianina/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
SEED=123

In [6]:
# загрузка датасета со всеми сэмплами
tweets = pd.read_csv('/datasets/toxic_comments.csv')

In [7]:
tweets.shape

(159292, 3)

In [8]:
tweets.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [9]:
# удаление ненужной колонки
tweets = tweets.drop(['Unnamed: 0'], axis=1)

In [10]:
tweets.shape

(159292, 2)

In [11]:
tweets.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [12]:
# количество пропущенных значений
tweets.isna().sum()

text     0
toxic    0
dtype: int64

In [13]:
# проверка на дисбаланс классов
tweets["toxic"].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [14]:
# выборка с 6000 признаками для создания эмбеддингов с помощью BERT 
tweets_bert = resample(
    tweets, n_samples=6000, random_state=SEED, stratify=tweets['toxic']
).reset_index(drop=True)

**Вывод:** я загрузила исходный датасет `tweets` со всеми твитами для работы с `TF-IDF` и `Word2vec` и отдельно создала выборку из 6000 твитов для дальнейшего извлечения эмбеддингов для предобученной `BERT` модели.

## Лемматизация и удаление стоп-слов

Для работы с TF-IDF и Word2vec необходимо подготовить признаки, а именно лемматизировать текст, удалить из него стоп-слова и привести все символы к нижнему регистру.

In [15]:
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [16]:
def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    return ' '.join(text.lower().split())

In [17]:
%%time

lemmatized_text = []
count = 0

for doc in lemmatizer.pipe(tweets['text'].apply(clear_text), n_process=-1):
    word_list = [
        word.lemma_ for word in doc
        if word.lemma_ not in stopwords
    ]
    lemmatized_text.append(' '.join(word_list))
    count += 1
    if count % 10000 == 0:
        print('Обработано', count, 'твитов')

Обработано 10000 твитов
Обработано 20000 твитов
Обработано 30000 твитов
Обработано 40000 твитов
Обработано 50000 твитов
Обработано 60000 твитов
Обработано 70000 твитов
Обработано 80000 твитов
Обработано 90000 твитов
Обработано 100000 твитов
Обработано 110000 твитов
Обработано 120000 твитов
Обработано 130000 твитов
Обработано 140000 твитов
Обработано 150000 твитов
CPU times: user 2min 51s, sys: 4.27 s, total: 2min 55s
Wall time: 2min 58s


In [18]:
tweets['text_lemma'] = lemmatized_text

In [19]:
tweets.head()

,text,toxic,text_lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation edit make username hardcore metall...
1,D'aww! He matches this background colour I'm s...,0,d'aww match background colour I seemingly stic...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I really try edit war guy constantly r...
3,"""\nMore\nI can't make any real suggestions on ...",0,I make real suggestion improvement I wonder se...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


## Разделение данных на выборки

In [20]:
target = tweets['toxic']
target_bert = tweets_bert['toxic']

In [21]:
# tfidf
features_tfidf_train, features_tfidf_test, target_tfidf_train, target_tfidf_test = train_test_split(
    tweets['text_lemma'], target, test_size=0.25
)


In [22]:
# word2vec
features_wordvec_train, features_wordvec_test, target_wordvec_train, target_wordvec_test = train_test_split(
    tweets['text_lemma'], target, test_size=0.25
)

In [23]:
# bert
features_bert_train, features_bert_test, target_bert_train, target_bert_test = train_test_split(
    tweets_bert['text'], target_bert, test_size=0.25
)

In [24]:
print(features_tfidf_train.shape)
print(target_tfidf_train.shape)
print(features_tfidf_test.shape)
print(target_tfidf_test.shape)

(119469,)
(119469,)
(39823,)
(39823,)


In [25]:
print(features_wordvec_train.shape)
print(target_wordvec_train.shape)
print(features_wordvec_test.shape)
print(target_wordvec_test.shape)

(119469,)
(119469,)
(39823,)
(39823,)


In [26]:
print(features_bert_train.shape)
print(target_bert_train.shape)
print(features_bert_test.shape)
print(target_bert_test.shape)

(4500,)
(4500,)
(1500,)
(1500,)


In [27]:
features_wordvec_train.head()

5439     maybe sentence could something like form w tor...
55140    I guess jfd try discourage get touch I via e m...
99008    sorry seem little harsh thing go little tit sp...
30599    favor hey still talk outstanding oppose wikipe...
3872     please stop add nonsense wikipedia consider va...
Name: text_lemma, dtype: object

## TF-IDF

Подход TF-IDF учитывает встречаемость слова не только в отдельном тексте (или твите), но и во всем корпусе текстов. 
TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.

TF = t/n, где t - количество употребления слова, а n — общее число слов в тексте.

IDF = log(D/d), где D - общее число текстов в корпусе, d - количество текстов, в которых это слово встречается.

`TFIDF = TF * IDF`

In [28]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(features_tfidf_train)

In [29]:
tfidf.shape

(119469, 127526)

In [30]:
# преобразование тестовой выборки
tfidf_test = tfidf_vectorizer.transform(features_tfidf_test)

In [31]:
tfidf_test.shape

(39823, 127526)

## Word2vec

Word embeddings - способ представления слов в виде векторов, при котором сохраняется семантическая связь между словами.

In [62]:
# токенизация текста
tokenized_tweet = features_wordvec_train.apply(lambda x: x.split())

In [63]:
tokenized_tweet

5439      [maybe, sentence, could, something, like, form...
55140     [I, guess, jfd, try, discourage, get, touch, I...
99008     [sorry, seem, little, harsh, thing, go, little...
30599     [favor, hey, still, talk, outstanding, oppose,...
3872      [please, stop, add, nonsense, wikipedia, consi...
                                ...                        
90930     [I, hope, right, way, reply, good, question, c...
157798    [push, advise, dig, source, c, e, e, fact, che...
80775     [since, ask, cite, bresloff, 's, involvement, ...
89265     [I, continue, edit, page, lock, formal, vote, ...
89325     [go, go, around, circle, wrong, I, already, ta...
Name: text_lemma, Length: 119469, dtype: object

In [64]:
model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            vector_size=300, 
            window=4, 
            min_count=2,                          
            sg = 1, 
            hs = 0,
            negative = 10,
            workers= 32,
            seed = SEED
) 

model_w2v.build_vocab(tokenized_tweet, progress_per=10000)
model_w2v.train(tokenized_tweet, total_examples= len(features_wordvec_train), epochs=20)

2022-11-15 14:36:06,742 : INFO : collecting all words and their counts
2022-11-15 14:36:06,743 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-11-15 14:36:06,796 : INFO : PROGRESS: at sentence #10000, processed 366311 words, keeping 28304 word types
2022-11-15 14:36:06,848 : INFO : PROGRESS: at sentence #20000, processed 731481 words, keeping 43376 word types
2022-11-15 14:36:06,900 : INFO : PROGRESS: at sentence #30000, processed 1093351 words, keeping 55587 word types
2022-11-15 14:36:06,951 : INFO : PROGRESS: at sentence #40000, processed 1460817 words, keeping 66315 word types
2022-11-15 14:36:07,004 : INFO : PROGRESS: at sentence #50000, processed 1830579 words, keeping 76174 word types
2022-11-15 14:36:07,060 : INFO : PROGRESS: at sentence #60000, processed 2207618 words, keeping 85344 word types
2022-11-15 14:36:07,114 : INFO : PROGRESS: at sentence #70000, processed 2572722 words, keeping 93229 word types
2022-11-15 14:36:07,167 : INFO : PROGRESS

(78626161, 88159220)

Можно посмотреть как работает модель Word2vec, для этого укажу слово, и модель вытянет из корпуса наиболее похожие слова:

In [65]:
model_w2v.wv.most_similar(positive="good")

[('eti', 0.5845963358879089),
 ('assumme', 0.5642857551574707),
 ('cabbynet', 0.5618889331817627),
 ('wikisunn', 0.5575671792030334),
 ('nsle', 0.5565122365951538),
 ('jimini', 0.5545695424079895),
 ('chllorine', 0.5499983429908752),
 ('mapfluger', 0.5456172227859497),
 ('mickwest', 0.5416241884231567),
 ('ubiq', 0.5394173860549927)]

Поскольку один твит - это набор слов, то необходимо получить один вектор из набора векторов, представляющих каждое слово в отдельности. Чтобы описать целое предложение буду использовать подход с вычислением среднего для всех векторов слов, входящих в твит.

In [66]:
# функция для вычисления "среднего" вектора
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv.get_vector(word).reshape((1, size))
            count += 1.
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

In [67]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 300)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet.iloc[i], 300)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(119469, 300)

In [68]:
# для тестовой выборки
tokenized_tweet_test = features_wordvec_test.apply(lambda x: x.split())

In [69]:
wordvec_arrays = np.zeros((len(tokenized_tweet_test), 300)) 
for i in range(len(tokenized_tweet_test)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet_test.iloc[i], 300)
wordvec_df_test = pd.DataFrame(wordvec_arrays)
wordvec_df_test.shape

(39823, 300)

## BERT. Создание эмбеддингов

In [41]:
model_class = transformers.BertModel
tokenizer_class = transformers.BertTokenizer
pretrained_weights = 'unitary/toxic-bert'

In [42]:
# загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Применю метод padding, чтобы после токенизации длины исходных текстов в корпусе были равными. Задам `max_len = 512`, чтобы учесть ограничение модели BERT.

In [43]:
max_len = 512

tokenized = features_bert_train.apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)
)      

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [44]:
display(padded.shape, attention_mask.shape)

(4500, 512)

(4500, 512)

Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, буду получать эмбеддинги батчами по 50 элементов.

In [45]:
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/91 [00:00<?, ?it/s]

In [46]:
# матрица признаков
features_bert = np.concatenate(embeddings)

In [47]:
features_bert[:2]

array([[-0.3431532 ,  0.8272309 ,  1.2637869 , ..., -0.52196896,
         0.16769446, -0.13926   ],
       [-0.6587919 , -0.45985788,  0.66098875, ..., -0.10696445,
         0.68867636,  0.00931725]], dtype=float32)

## Подбор гиперпараметров

- **LogicticRegression для TF-IDF**

In [53]:
def objective(trial):
    
    param = {
        'solver': trial.suggest_categorical('solver', ['liblinear']),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'C': trial.suggest_float('C', 0.01, 4.0, log=True),
        'tol': trial.suggest_float('tol', 0.0001, 0.01, log=True)
    }
    
    lr = LogisticRegression(**param, random_state=SEED, n_jobs=-1)
    lr.fit(tfidf, target_tfidf_train)
    f1 = f1_score(target_tfidf_test, lr.predict(tfidf_test))
    return f1

In [54]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-11-15 14:24:16,203] A new study created in memory with name: no-name-c810e600-3140-47e8-af10-fb5bd9762189
/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
[I 2022-11-15 14:24:16,720] Trial 0 finished with value: 0.7825497900013547 and parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 2.166468352733118, 'tol': 0.009721680092828781}. Best is trial 0 with value: 0.7825497900013547.
/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
[I 2022-11-15 14:24:17,621] Trial 1 finished with value: 0.4966048816296568 and parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.08371368106482249, 'tol': 0.0

CPU times: user 2min 35s, sys: 1min 42s, total: 4min 18s
Wall time: 1min 17s


In [55]:
study.best_trial

FrozenTrial(number=82, values=[0.7837692100296575], datetime_start=datetime.datetime(2022, 11, 15, 14, 25, 19, 407057), datetime_complete=datetime.datetime(2022, 11, 15, 14, 25, 20, 228719), params={'solver': 'liblinear', 'penalty': 'l1', 'C': 2.5514102655758655, 'tol': 0.0015554452120822276}, distributions={'solver': CategoricalDistribution(choices=('liblinear',)), 'penalty': CategoricalDistribution(choices=('l1', 'l2')), 'C': FloatDistribution(high=4.0, log=True, low=0.01, step=None), 'tol': FloatDistribution(high=0.01, log=True, low=0.0001, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=82, state=TrialState.COMPLETE, value=None)

params={'solver': 'liblinear', 'penalty': 'l1', 'C': 2.5514102655758655, 'tol': 0.0015554452120822276}

- **LogicticRegression для Word2vec**

In [70]:
def objective(trial):
    
    param = {
        'solver': trial.suggest_categorical('solver', ['liblinear']),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'C': trial.suggest_float('C', 0.01, 4.0, log=True),
        'tol': trial.suggest_float('tol', 0.0001, 0.01, log=True)
    }
    
    lr = LogisticRegression(**param, random_state=SEED, n_jobs=-1)
    lr.fit(wordvec_df, target_wordvec_train)
    f1 = f1_score(target_wordvec_test, lr.predict(wordvec_df_test))
    return f1

In [71]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-11-15 14:41:22,790] A new study created in memory with name: no-name-269cfc12-8be0-48b6-9a65-49e8d377a135
/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
[I 2022-11-15 14:41:36,783] Trial 0 finished with value: 0.6944770857814336 and parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.38769253038506113, 'tol': 0.0010669889836632633}. Best is trial 0 with value: 0.6944770857814336.
/home/troflianina/.pyenv/versions/3.9.5/envs/nlp/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
[I 2022-11-15 14:41:42,227] Trial 1 finished with value: 0.6956393635827932 and parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.24970963467529578, 'tol': 

CPU times: user 17min 22s, sys: 1min 21s, total: 18min 43s
Wall time: 16min 47s


In [72]:
study.best_trial

FrozenTrial(number=12, values=[0.7025767942932013], datetime_start=datetime.datetime(2022, 11, 15, 14, 43, 44, 202228), datetime_complete=datetime.datetime(2022, 11, 15, 14, 43, 49, 466216), params={'solver': 'liblinear', 'penalty': 'l2', 'C': 3.6564734727044144, 'tol': 0.00010105309302122329}, distributions={'solver': CategoricalDistribution(choices=('liblinear',)), 'penalty': CategoricalDistribution(choices=('l1', 'l2')), 'C': FloatDistribution(high=4.0, log=True, low=0.01, step=None), 'tol': FloatDistribution(high=0.01, log=True, low=0.0001, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=12, state=TrialState.COMPLETE, value=None)

params={'solver': 'liblinear', 'penalty': 'l2', 'C': 3.6564734727044144, 'tol': 0.00010105309302122329}

## Обучение

Обучу модели логистической регрессии и CatBoost на признаках, полученных с помощью трех разных подходов.

### TFIDF

In [73]:
"""Логистическая регрессия"""

params = {
    'solver': 'liblinear', 
    'penalty': 'l1', 
    'C': 2.5514102655758655, 
    'tol': 0.0015554452120822276
}

model_logistic_regression = LogisticRegression(**params, random_state=SEED)
scores = cross_val_score(
    model_logistic_regression, 
    tfidf, 
    target_tfidf_train, 
    cv=5,
    scoring='f1'
)
final_score = sum(scores) / len(scores)
 
print(f'F1-мера на обучающей выборке: {final_score}')

F1-мера на обучающей выборке: 0.7813524302689252


**F1-мера на обучающей выборке: 0.7813524302689252**

In [ ]:
"""CatBoost"""

model_catboost = CatBoostClassifier(
    iterations=400, eval_metric='F1', verbose=0, random_state=SEED
)

scores = cross_val_score(
    model_catboost, 
    tfidf, 
    target_tfidf_train, 
    cv=5, 
    scoring='f1'
)
final_score = sum(scores) / len(scores)
 
print(f'F1-мера на обучающей выборке: {final_score}')

**F1-мера на обучающей выборке: 0.7469776675913491**

### Word2vec

In [74]:
"""Логистическая регрессия"""

params = {
    'solver': 'liblinear', 
    'penalty': 'l2', 
    'C': 3.6564734727044144, 
    'tol': 0.00010105309302122329
}

model_logistic_regression = LogisticRegression(**params, random_state=SEED)
scores = cross_val_score(
    model_logistic_regression, 
    wordvec_df, 
    target_wordvec_train, 
    cv=5,
    scoring='f1'
)
final_score = sum(scores) / len(scores)
 
print(f'F1-мера на обучающей выборке: {final_score}')

F1-мера на обучающей выборке: 0.7066265008919472


**F1-мера на обучающей выборке: 0.7066265008919472**

In [77]:
"""CatBoost"""

model_catboost = CatBoostClassifier(
    iterations=400, eval_metric='F1', verbose=0, random_state=SEED
)

scores = cross_val_score(
    model_catboost, 
    wordvec_df, 
    target_wordvec_train,
    cv=5, 
    scoring='f1'
)
final_score = sum(scores) / len(scores)
 
print(f'F1-мера на обучающей выборке: {final_score}')

F1-мера на обучающей выборке: 0.7529563557083792


**F1-мера на обучающей выборке: 0.7529563557083792**

### BERT

In [78]:
"""Логистическая регрессия"""

model_logistic_regression = LogisticRegression(max_iter=1000, random_state=SEED)
scores = cross_val_score(
    model_logistic_regression, 
    features_bert, 
    target_bert_train, 
    cv=5, 
    scoring='f1'
)
final_score = sum(scores) / len(scores)
 
print(f'F1-мера на обучающей выборке: {final_score}')

F1-мера на обучающей выборке: 0.9449107743245133


**F1-мера на обучающей выборке: 0.9449107743245133**

In [79]:
"""CatBoost"""

model_catboost = CatBoostClassifier(
    iterations=400, eval_metric='F1', verbose=0, random_state=SEED
)

scores = cross_val_score(
    model_catboost, 
    features_bert, 
    target_bert_train, 
    cv=5, 
    scoring='f1'
)
final_score = sum(scores) / len(scores)
 
print(f'F1-мера на обучающей выборке: {final_score}')

F1-мера на обучающей выборке: 0.9479702805416708


**F1-мера на обучающей выборке: 0.9479702805416708**

**Вывод:** по итогами оценки разных подходов представления текстовых признаков лучшим оказался метод `Bert` и обученная на данных эмбеддингах модель `CatBoost` с результатом на обучающей выборке ключевой метрики F1-меры = 0.948.

## Проверка на тесте лучшей модели и подхода

## BERT

In [81]:
max_len = 512

tokenized = features_bert_test.apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)
)      

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [82]:
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/31 [00:00<?, ?it/s]

In [83]:
features_test_bert = np.concatenate(embeddings)

In [84]:
%%time

model_catboost = CatBoostClassifier(
    iterations=400, eval_metric='F1', verbose=0, random_state=SEED
)

model_catboost.fit(features_bert, target_bert_train)
predicted = model_catboost.predict(features_test_bert)
score = f1_score(predicted, target_bert_test)
 
print(f'F1-мера на тестовой выборке: {score:.4f}')

F1-мера на тестовой выборке: 0.9337
CPU times: user 4min 36s, sys: 4.7 s, total: 4min 41s
Wall time: 24.2 s


**F1-мера на тестовой выборке: 0.9337**

## Выводы

Обучила модели машинного обучения на признаках, полученных тремя разными способами:
- TF-IDF;
- Word2vec;
- BERT.

Лучшим стал способ получения эмбеддингов с помощью **BERT-модели** для определения токсичности текста. По полученным эмбеддингам я обучила модели из классического ML, лучшей стала модель **CatBoost с результатом F1-меры на обучающей выборке - 0.948 и на тесте - 0.9337**.